In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:95% !important;}</style>"))
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
#from seq_generator.query_generator import *
#from seq_generator.mismatch_generator import *
from encoder_decoder.seq_encoder_decoder import *
#import encoder_decoder.seq_encoder_decoder
from seq_generator.data_generator import *
from job_script.cas_offinder_job import *

In [4]:
import numpy as np

In [5]:
job_name = 'mis_save_test'
path = '.'

In [6]:
tmp_data = generate_8_nC2_data(
    num_set = 3,
    threshold = 7,
    log = True,
    job_name = job_name,
    save_path = path,
)

In [7]:
tmp_data.ontarget_data

array(['CTCATACAAACATGTAAATT', 'GATTTTTATATTTTGGCCTC',
       'TCCAAACGAGGCGCTTAGTG', 'TAGGCTTCGCCCCTGTAGCC',
       'TGACGCTAAGCAAGTATTAT', 'GTACCCCATTGCGACGCCGG',
       'GGGTAGCACTCACCATATGA', 'CCCGCCCCGCGGTGTGGCGA',
       'TTAAAAAAATTAGTTTGCAT', 'AAAAATAAATACAGGGCCCC',
       'AGTATTAACGGCAGCACATA', 'CTAGACTGTAGGCTGCGCCG',
       'CCGCGGGAATATGTTTTACT', 'TCTGAGCAAAATATGGCGGC',
       'CGGGGCGGCCCCAATAATCC', 'GGCCGCCGTCCGGGAGGCCG',
       'TAATCTCTAGGGTTGATAAT', 'TTATATATTTTAATGGCGCT',
       'GATATTTCGCCTGGAGTTAC', 'ACTAGGTGCGGCCGCCAGGG',
       'CCACAGGGACAACTTTTTTT', 'AACGGGGTATTAAGCGCGCG',
       'GAGGCGCAAGGGCCTTCTCT', 'GCGCCCCGGCGCGAGGGGAC'], dtype='<U20')

In [8]:
tmp_data.mis_1_data.shape

(24, 60)

In [9]:
tmp_data.mis_2_data.shape

(24, 570)

In [10]:
tmp_data.mis_2_dis_data.shape

(24, 80)

In [36]:
ontarget_list = tmp_data.ontarget_data
offtarget_list = tmp_data.mis_2_data
pd_data_list = []
for _ontarget, _offtarget_list in zip(ontarget_list, offtarget_list):
    for _offtarget in _offtarget_list:
        _mismatch_num = mismatch_calculator.seq_mismatch_count(_ontarget,_offtarget)
        _mismatch_info = mismatch_calculator.seq_mismatch_info_str(_ontarget, _offtarget)
        pd_data_list.append([_ontarget,_offtarget,_mismatch_num, _mismatch_info])

In [37]:
df_columns = ['on-target','off-target','number of mismatch','mismatch info']
df_dtype = {
    'on-target' : 'category',
    'off-target' : 'category',
    'number of mismatch' : 'int8',
}
pd_data_df = pd.DataFrame(
    pd_data_list,
    columns=df_columns,
).astype(df_dtype)

In [38]:
pd_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13680 entries, 0 to 13679
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   on-target           13680 non-null  category
 1   off-target          13680 non-null  category
 2   number of mismatch  13680 non-null  int8    
 3   mismatch info       13680 non-null  object  
dtypes: category(2), int8(1), object(1)
memory usage: 908.1+ KB


In [39]:
pd_data_df.head()

,on-target,off-target,number of mismatch,mismatch info
0,CTCATACAAACATGTAAATT,AGCATACAAACATGTAAATT,2,"0:C>A,1:T>G"
1,CTCATACAAACATGTAAATT,GGCATACAAACATGTAAATT,2,"0:C>G,1:T>G"
2,CTCATACAAACATGTAAATT,TGCATACAAACATGTAAATT,2,"0:C>T,1:T>G"
3,CTCATACAAACATGTAAATT,ATTATACAAACATGTAAATT,2,"0:C>A,2:C>T"
4,CTCATACAAACATGTAAATT,GTTATACAAACATGTAAATT,2,"0:C>G,2:C>T"


In [ ]:
#run_cas_offinder(
#    file_path = f"{path}/{job_name}/run_{job_name}_script.sh",
#    log = True
#)

In [ ]:
tmp_df = analysis_cas_offinder_result(
    output_file_path = f"{path}/{job_name}/{job_name}_output.txt",
    query_text_path = f"{path}/{job_name}/query_{job_name}.txt",
)

In [ ]:
tmp_df

In [ ]:
def query_seq_generate(job_name, path='./', method=generate_8_nC2_data):
    job_path = f"{path}/{job_name}"
    cas_offinder_script_path = f"{job_path}/run_{job_name}_script.sh"
    output_file_path = f"{job_path}/{job_name}_output.txt"
    query_file_path = f"{job_path}/{job_name}_query.txt"
    data = method(
        num_set = 3,
        threshold = 7,
        log = True,
        job_name = job_name,
        save_path = path,
    )
    make_cas_offinder_script(
        query_seq_list = data.query_data,
        job_name = job_name,
        job_path = job_path,
    )
    run_cas_offinder(
        file_path = cas_offinder_script_path,
        log = True,
    )
    result_df = analysis_cas_offinder_result(
        output_file_path = output_file_path,
        query_text_path = query_file_path,
    )
    return result_df

In [ ]:
query_seq_generate(
    job_name = "pipeline_test",
    path = './test',
    method = generate_8_nC2_data,
)

In [ ]:
tmp_data = generate_8_nC2_data(
    num_set = 1,
    threshold = 7,
    log = True,
    job_name = 'test',
    save_path = '.'
)

In [ ]:
tmp_query = tmp_data.query_data
tmp_query

In [ ]:
make_cas_offinder_script(
    query_seq_list = tmp_query,
    job_name = 'test_job',
    job_path = f'./test',
)

In [ ]:
seq_list = tmp_query
batch_type_code_seq_list = []
for _seq in seq_list:
    _type_code = query_type_code_encoder.seq_type_code_encoder(_seq)
    batch_type_code_seq_list.append((_type_code,_seq))

In [ ]:
import pandas as pd

In [ ]:
analysis_cas_offinder_result(
    output_file_path='./test/test_job_out.txt',
    query_text_path='./test/test_query.txt',
    save_csv=True,
)

In [ ]:
command = ["bash",'../test/run_test_job_script.sh']
cas_offinder_run = subprocess.Popen(
    command,
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True,
)

In [ ]:
output, log = cas_offinder_run.communicate()

In [ ]:
print(output)

In [ ]:
print(log)

In [ ]:
_seq_1 = 'ACGTAA'
_seq_2 = 'ACGTAA'
_mismatch_seq_list = ['AAGTAA','ACATAA','ACGAAA','ACGTCA','ACGTGG']

In [ ]:
np.sum(np.clip(one_hot_encoder.seq_encoder(_seq_1) - one_hot_encoder.seq_encoder(_seq_2),0,None))

In [ ]:
mismatch_calculator.seq_mismatch_count_list(_seq_1,_mismatch_seq_list)

In [15]:
ontarget_text_file_path = '/root/volume/q-seq-maker/data/other_paper_data/KHB_Cpf1_1.txt'


In [16]:
with open(ontarget_text_file_path,'r') as f:
    data = f.read().splitlines()

In [19]:
data

['#KHB_Cpf1_1',
 'CTCCGCCgCACCTCCCCCAGCCC',
 'CTaCGCCCCACCTCCCCCAGCCC',
 'CTCCGCCCCACCTCCCCCAtCCC',
 'CgCCGCCCCACCTCCCCCAGCCC',
 'CTCCGCCCCACCTCtCCCAGCCC',
 'CTCgGCCCCACCTCCCCCAGCCC',
 'CTCCGCCCCACCTCCtCCAGCCC',
 'CTCCGCCtCACCTCCCCCAGCCC',
 'CTCCGCCCCACCTCCCCaAGCCC',
 'CTCCGtCCCACCTCCCCCAGCCC',
 'CTCCGCCCCACCTCCCCCtGCCC',
 'CTCCaCCCCACCTCCCCCAGCCC',
 'CTCCGCCCCACCTCCCaCAGCCC',
 'CTCCGCCCCACaTCCCCCAGCCC',
 'CTCCGCCCaACCTCCCCCAGCCC',
 'CTCCGCaCCACCTCCCCCAGCCC',
 'CTCCGCCCCACCTCCCCCAGCCt',
 'CTCCGCCCCgCCTCCCCCAGCCC',
 'CTCCGCCCCACCTCCaCCAGCCC',
 'CTtCGCCCCACCTCCCCCAGCCC',
 'CTCCGCCCCACCTCCCCCAGtCC',
 'CTCCGCCCgACCTCCCCCAGCCC',
 'CTCCGCCCCAtCTCCCCCAGCCC',
 'CTCCGCCCtACCTCCCCCAGCCC',
 'CTCCGCCCCACCTCCCCgAGCCC',
 'CTCCGCCCCACCTCCCgCAGCCC',
 'CTCCtCCCCACCTCCCCCAGCCC',
 'CTCCGCCCCACCTgCCCCAGCCC',
 'CTCCGaCCCACCTCCCCCAGCCC',
 'CTCCGCCCCACCaCCCCCAGCCC',
 'CTCCGgCCCACCTCCCCCAGCCC',
 'CTCCGCCCCACCTCCCtCAGCCC',
 'gTCCGCCCCACCTCCCCCAGCCC',
 'CTCCGCCCCACCTCCCCCAGCaC',
 'CTCCGCCaCACCTCCCCCAGCCC',
 'CT

In [34]:
%timeit data[0].find('#')
%timeit '#' in data[0]
%timeit p.findall(data[0])

112 ns ± 1.37 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
42.2 ns ± 0.293 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
181 ns ± 0.327 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [37]:
data_list = []
for _line in data:
    if '#' in _line:
        continue
    else:
        data_list.append(_line.upper())

In [38]:
data_list

['CTCCGCCGCACCTCCCCCAGCCC',
 'CTACGCCCCACCTCCCCCAGCCC',
 'CTCCGCCCCACCTCCCCCATCCC',
 'CGCCGCCCCACCTCCCCCAGCCC',
 'CTCCGCCCCACCTCTCCCAGCCC',
 'CTCGGCCCCACCTCCCCCAGCCC',
 'CTCCGCCCCACCTCCTCCAGCCC',
 'CTCCGCCTCACCTCCCCCAGCCC',
 'CTCCGCCCCACCTCCCCAAGCCC',
 'CTCCGTCCCACCTCCCCCAGCCC',
 'CTCCGCCCCACCTCCCCCTGCCC',
 'CTCCACCCCACCTCCCCCAGCCC',
 'CTCCGCCCCACCTCCCACAGCCC',
 'CTCCGCCCCACATCCCCCAGCCC',
 'CTCCGCCCAACCTCCCCCAGCCC',
 'CTCCGCACCACCTCCCCCAGCCC',
 'CTCCGCCCCACCTCCCCCAGCCT',
 'CTCCGCCCCGCCTCCCCCAGCCC',
 'CTCCGCCCCACCTCCACCAGCCC',
 'CTTCGCCCCACCTCCCCCAGCCC',
 'CTCCGCCCCACCTCCCCCAGTCC',
 'CTCCGCCCGACCTCCCCCAGCCC',
 'CTCCGCCCCATCTCCCCCAGCCC',
 'CTCCGCCCTACCTCCCCCAGCCC',
 'CTCCGCCCCACCTCCCCGAGCCC',
 'CTCCGCCCCACCTCCCGCAGCCC',
 'CTCCTCCCCACCTCCCCCAGCCC',
 'CTCCGCCCCACCTGCCCCAGCCC',
 'CTCCGACCCACCTCCCCCAGCCC',
 'CTCCGCCCCACCACCCCCAGCCC',
 'CTCCGGCCCACCTCCCCCAGCCC',
 'CTCCGCCCCACCTCCCTCAGCCC',
 'GTCCGCCCCACCTCCCCCAGCCC',
 'CTCCGCCCCACCTCCCCCAGCAC',
 'CTCCGCCACACCTCCCCCAGCCC',
 'CTCCGCCCCACCTCCGCC

In [36]:
data[1].upper()

'CTCCGCCGCACCTCCCCCAGCCC'